In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Load dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Target classes (0, 1, 2)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use long for multi-class
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [4]:
class AdvancedANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AdvancedANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Batch normalization
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # Dropout to prevent overfitting
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [5]:
input_size = X.shape[1]
hidden_size = 64
output_size = len(set(y))  # Number of classes

model = AdvancedANN(input_size, hidden_size, output_size)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [6]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    epoch_loss = 0

    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Validation
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)  # Predicted class
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {accuracy:.4f}")


Epoch 1/50, Loss: 5.5176, Validation Accuracy: 0.8667
Epoch 2/50, Loss: 3.4080, Validation Accuracy: 0.8333
Epoch 3/50, Loss: 2.7415, Validation Accuracy: 0.9333
Epoch 4/50, Loss: 2.6987, Validation Accuracy: 0.9333
Epoch 5/50, Loss: 2.1589, Validation Accuracy: 0.9333
Epoch 6/50, Loss: 2.2098, Validation Accuracy: 0.9333
Epoch 7/50, Loss: 1.9831, Validation Accuracy: 0.9333
Epoch 8/50, Loss: 2.0813, Validation Accuracy: 0.9333
Epoch 9/50, Loss: 3.0610, Validation Accuracy: 0.9333
Epoch 10/50, Loss: 3.4105, Validation Accuracy: 0.9667
Epoch 11/50, Loss: 1.9262, Validation Accuracy: 1.0000
Epoch 12/50, Loss: 2.8145, Validation Accuracy: 0.9333
Epoch 13/50, Loss: 1.5797, Validation Accuracy: 0.9667
Epoch 14/50, Loss: 2.1260, Validation Accuracy: 0.9667
Epoch 15/50, Loss: 1.8057, Validation Accuracy: 0.9667
Epoch 16/50, Loss: 4.6925, Validation Accuracy: 0.9667
Epoch 17/50, Loss: 2.3212, Validation Accuracy: 0.9667
Epoch 18/50, Loss: 2.0916, Validation Accuracy: 0.9667
Epoch 19/50, Loss: 

In [16]:
# Test a single sample
sample = torch.tensor([X_test[4]], dtype=torch.float32).to(device)
model.eval()
with torch.no_grad():
    output = model(sample)
    _, prediction = torch.max(output, 1)
    print(f"Predicted Class: {prediction.item()}, Actual Class: {y_test[0]}")


Predicted Class: 1, Actual Class: 1
